 CSV que combina Metadata con Finding Annotations. Si hay una fila repetida en Finding Annotations, duplica una en Metadata para no perder esa información.

In [1]:
import pandas as pd

# Ruta a los archivos CSV
finding_annotations_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/original/Vindrmammo/finding_annotations.csv'
metadata_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/original/Vindrmammo/metadata.csv'

# Cargar los CSV en dataframes de pandas
finding_annotations = pd.read_csv(finding_annotations_path)
metadata = pd.read_csv(metadata_path)

# Hacer la unión de los dataframes usando 'image_id' en finding_annotations y 'SOP Instance UID' en metadata
# Primero vamos a asegurarnos que las columnas estén correctamente nombradas y con el tipo de datos correcto

# Verificar si hay columnas extra que podamos eliminar para hacer más limpia la unión
# La columna 'image_id' en finding_annotations tiene que coincidir con 'SOP Instance UID' en metadata
merged_df = pd.merge(finding_annotations, metadata, how='outer', left_on='image_id', right_on='SOP Instance UID')

# Verificar si hay duplicados en 'image_id' y manejarlos creando filas duplicadas de 'SOP Instance UID'
# Duplicamos las filas de image_id repetidos y mantenemos toda la información
duplicated_image_ids = merged_df[merged_df.duplicated(subset='image_id', keep=False)]

# Imprimir cuántos duplicados se encontraron
print(f"Se encontraron {len(duplicated_image_ids)} filas con image_id duplicados")

# Guardar el dataframe combinado en un nuevo CSV
output_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv'
merged_df.to_csv(output_csv_path, index=False)

print(f"Archivo combinado guardado en: {output_csv_path}")


Se encontraron 824 filas con image_id duplicados
Archivo combinado guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv


Ahora creamos otro CSV que contenga solo las categorías "Mass" o "Suscipious Calcifications" en la columna finding annotations.
No 2 categorías o más , ni mass y suspicious calcifications juntas. Esto es porque queremos que el sistema reconozca si es una massa o calcificacion benigna, sospechosa o maligna y al agregarle más hallazgos a una imagen esto la condiciona o aumenta las probabilidades de que sea maligna.

In [2]:
import pandas as pd

# Ruta al archivo CSV combinado
combined_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv'

# Cargar el CSV combinado
combined_df = pd.read_csv(combined_csv_path)

# Filtrar las filas que tengan exactamente una categoría en 'finding_categories' que sea "Mass" o "Suspicious Calcification"
def filter_mass_or_calcification(categories):
    # Verificar si es una lista de una sola categoría y si esa categoría es "Mass" o "Suspicious Calcification"
    categories_list = eval(categories) if isinstance(categories, str) else categories
    return len(categories_list) == 1 and categories_list[0] in ['Mass', 'Suspicious Calcification']

filtered_df = combined_df[combined_df['finding_categories'].apply(filter_mass_or_calcification)]

# Guardar el nuevo CSV filtrado
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications.csv'
filtered_df.to_csv(filtered_csv_path, index=False)

print(f"CSV filtrado guardado en: {filtered_csv_path}")


CSV filtrado guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications.csv


Ahora creamos un CSV con nueva columna image_name para diferenciar las imagenes que tienen varios image_id

In [3]:
import pandas as pd

# Ruta al CSV filtrado
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications.csv'

# Cargar el CSV filtrado
filtered_csv = pd.read_csv(filtered_csv_path)

# Crear una nueva columna 'image_name' que sea una copia inicial del 'SOP Instance UID'
filtered_csv['image_name'] = ''

# Agrupar por 'SOP Instance UID' y contar las repeticiones
image_counts = filtered_csv.groupby('SOP Instance UID').cumcount()

# Asignar el nuevo valor en 'image_name' con el formato {SOP Instance UID}_{n}
for index, row in filtered_csv.iterrows():
    sop_uid = row['SOP Instance UID']
    count = image_counts[index]
    filtered_csv.at[index, 'image_name'] = f"{sop_uid}_{count}"

# Guardar el nuevo CSV con la columna 'image_name'
new_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_with_image_names.csv'
filtered_csv.to_csv(new_csv_path, index=False)

print(f"Nuevo CSV guardado en: {new_csv_path}")


Nuevo CSV guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_with_image_names.csv


Ahora ya tenemos el csv que necesitamos.

Ahora creamos un csv que contenga no solo mass y calcification si no todas las categorías menos "No Finding"

In [1]:
import pandas as pd
import ast  # Importar ast para evaluar literales de forma segura

# Ruta al archivo CSV combinado
combined_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv'

# Cargar el CSV combinado
combined_df = pd.read_csv(combined_csv_path)

# Lista de categorías a incluir
categories_to_include = [
    'Mass',
    'Suspicious Calcification',
    'Focal Asymmetry',
    'Architectural Distortion',
    'Asymmetry',
    'Suspicious Lymph Node',
    'Skin Thickening',
    'Global Asymmetry',
    'Nipple Retraction',
    'Skin Retraction'
]

# Función para filtrar filas con exactamente una categoría de interés
def filter_single_category(categories):
    # Verificar si 'categories' es una cadena que representa una lista
    if isinstance(categories, str):
        try:
            categories_list = ast.literal_eval(categories)
        except (ValueError, SyntaxError):
            # Si no se puede evaluar, retornamos False
            return False
    else:
        categories_list = categories

    # Verificar que sea una lista con exactamente una categoría de interés
    return (
        isinstance(categories_list, list) and
        len(categories_list) == 1 and
        categories_list[0] in categories_to_include
    )

# Aplicar el filtro al DataFrame
filtered_df = combined_df[combined_df['finding_categories'].apply(filter_single_category)]

# Guardar el nuevo CSV filtrado
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications_and_others.csv'
filtered_df.to_csv(filtered_csv_path, index=False)

print(f"CSV filtrado guardado en: {filtered_csv_path}")


CSV filtrado guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications_and_others.csv


Añadimos la columna image_name

In [2]:
import pandas as pd

# Ruta al CSV filtrado (resultado del paso anterior)
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/filtered_mass_calcifications_and_others.csv'

# Cargar el CSV filtrado
filtered_csv = pd.read_csv(filtered_csv_path)

# Crear una nueva columna 'image_name' que sea una copia inicial del 'SOP Instance UID'
filtered_csv['image_name'] = ''

# Agrupar por 'SOP Instance UID' y contar las repeticiones
image_counts = filtered_csv.groupby('SOP Instance UID').cumcount()

# Asignar el nuevo valor en 'image_name' con el formato {SOP Instance UID}_{n}
for index, row in filtered_csv.iterrows():
    sop_uid = row['SOP Instance UID']
    count = image_counts[index]
    filtered_csv.at[index, 'image_name'] = f"{sop_uid}_{count}"

# Guardar el nuevo CSV con la columna 'image_name' y el nombre de archivo ajustado
new_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/others_with_image_names.csv'
filtered_csv.to_csv(new_csv_path, index=False)

print(f"Nuevo CSV guardado en: {new_csv_path}")


Nuevo CSV guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/others_with_image_names.csv


### Primeras 2500 no finding y otras anotaciones ( 1 sola por imágenes)    

In [5]:
import pandas as pd
import ast

# Ruta al archivo CSV combinado
combined_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/combined_annotations_metadata.csv'

# Cargar el CSV combinado
try:
    combined_df = pd.read_csv(combined_csv_path)
    print("CSV cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo en la ruta '{combined_csv_path}' no se encontró.")
    exit(1)
except pd.errors.ParserError as e:
    print(f"Error al parsear el CSV: {e}")
    exit(1)

# Función para convertir la cadena de categorías a una lista de forma segura
def parse_categories(categories):
    if isinstance(categories, str):
        try:
            # Usa ast.literal_eval para mayor seguridad
            return ast.literal_eval(categories)
        except (ValueError, SyntaxError):
            # Si falla, asume que es una única categoría
            return [categories]
    elif isinstance(categories, list):
        return categories
    else:
        # Maneja otros tipos de datos inesperados
        return []

# Aplicar la función para crear una nueva columna con listas de categorías
combined_df['categories_list'] = combined_df['finding_categories'].apply(parse_categories)

# Filtrar filas que tengan exactamente una categoría
single_category_df = combined_df[combined_df['categories_list'].apply(len) == 1].copy()
print(f"Filas con exactamente una categoría: {single_category_df.shape[0]}")

# Crear una nueva columna 'single_category' con la categoría única en minúsculas y sin espacios
single_category_df['single_category'] = single_category_df['categories_list'].str[0].str.lower().str.strip()

# Verificar las categorías únicas después de la normalización
unique_categories = single_category_df['single_category'].unique()
print("Categorías únicas después de la normalización:")
print(unique_categories)

# Verificar el número de filas "no finding" antes de limitar
total_no_finding = single_category_df[single_category_df['single_category'] == 'no finding'].shape[0]
print(f"Total de filas 'no finding' antes de filtrar: {total_no_finding}")

# Filtrar filas con "no finding" y tomar solo las primeras 2500
no_finding_df = single_category_df[single_category_df['single_category'] == 'no finding'].head(2500).copy()
filtered_no_finding = no_finding_df.shape[0]
print(f"Filas 'no finding' incluidas en el CSV final: {filtered_no_finding}")

# Filtrar las filas que tienen exactamente una categoría y que no sean "no finding"
other_categories_df = single_category_df[single_category_df['single_category'] != 'no finding'].copy()
other_categories_count = other_categories_df.shape[0]
print(f"Total de filas en otras categorías: {other_categories_count}")

# Combinar las demás categorías con las filas seleccionadas de "no finding"
filtered_df = pd.concat([other_categories_df, no_finding_df], ignore_index=True)
print(f"Total de filas después de combinar: {filtered_df.shape[0]}")

# Opcional: Mezclar el dataframe para distribuir aleatoriamente las filas
filtered_df = filtered_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("DataFrame mezclado aleatoriamente.")

# Contar las ocurrencias de cada categoría en el DataFrame filtrado
category_counts = filtered_df['single_category'].value_counts()
print("Distribución de categorías en el CSV filtrado:")
print(category_counts)

# Verificar que solo hay 2500 filas "no finding"
if category_counts.get('no finding', 0) > 2500:
    print("Advertencia: Hay más de 2500 filas 'no finding' en el CSV filtrado.")
else:
    print("Correcto: Las filas 'no finding' están limitadas a 2500.")

# Eliminar las columnas auxiliares si ya no se necesitan
filtered_df = filtered_df.drop(columns=['categories_list', 'single_category'])
print("Columnas auxiliares eliminadas.")

# Ruta para guardar el CSV filtrado
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/4categories.csv'

# Guardar el nuevo CSV filtrado
try:
    filtered_df.to_csv(filtered_csv_path, index=False)
    print(f"CSV filtrado guardado en: {filtered_csv_path}")
except Exception as e:
    print(f"Error al guardar el CSV: {e}")


CSV cargado exitosamente.
Filas con exactamente una categoría: 20312
Categorías únicas después de la normalización:
['no finding' 'mass' 'suspicious calcification' 'focal asymmetry'
 'architectural distortion' 'asymmetry' 'skin thickening'
 'suspicious lymph node' 'global asymmetry' 'nipple retraction'
 'skin retraction']
Total de filas 'no finding' antes de filtrar: 18232
Filas 'no finding' incluidas en el CSV final: 2500
Total de filas en otras categorías: 2080
Total de filas después de combinar: 4580
DataFrame mezclado aleatoriamente.
Distribución de categorías en el CSV filtrado:
single_category
no finding                  2500
mass                        1123
suspicious calcification     402
focal asymmetry              232
architectural distortion      95
asymmetry                     90
suspicious lymph node         57
skin thickening               38
global asymmetry              24
nipple retraction             12
skin retraction                7
Name: count, dtype: int64
Corr

In [6]:
#Añadimos la columna image name

In [7]:
import pandas as pd

# Ruta al CSV filtrado (resultado del paso anterior)
filtered_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/4categories.csv'

# Cargar el CSV filtrado
filtered_csv = pd.read_csv(filtered_csv_path)

# Crear una nueva columna 'image_name' que sea una copia inicial del 'SOP Instance UID'
filtered_csv['image_name'] = ''

# Agrupar por 'SOP Instance UID' y contar las repeticiones
image_counts = filtered_csv.groupby('SOP Instance UID').cumcount()

# Asignar el nuevo valor en 'image_name' con el formato {SOP Instance UID}_{n}
for index, row in filtered_csv.iterrows():
    sop_uid = row['SOP Instance UID']
    count = image_counts[index]
    filtered_csv.at[index, 'image_name'] = f"{sop_uid}_{count}"

# Guardar el nuevo CSV con la columna 'image_name' y el nombre de archivo ajustado
new_csv_path = '/Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/4categories_with_image_names.csv'
filtered_csv.to_csv(new_csv_path, index=False)

print(f"Nuevo CSV guardado en: {new_csv_path}")


Nuevo CSV guardado en: /Volumes/m2/Memoria/Code/PMM/VinDr-Mammo-Preprocessing/data/processed/csv/4categories_with_image_names.csv
